## Logistic Regression for the MNIST dataset

In this notebook we will continue using Pytorch. Now, we will create a shallow model (the logistic regression), plus the full training pipeline in order to train and evaluate this model over the MNIST dataset.

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Next, we will define some hyperparameters and config options

In [50]:
# Set the random seed, so the experiment is reproducible
seed = 0
torch.manual_seed(seed)
# For the moment, we will just train on CPU, so no cuda
use_cuda = False
# We use a batch size of 32 examples while training, and 1000 while testing
batch_size = 32
test_batch_size = 1000
# We will use SGD with a momentum term
momentum = 0.5
# The learning rate
lr = 0.01
# The number of epochs
epochs = 10
# The size of the input. MNIST are greyscale images, 28x28 pixels each
im_size = 28*28

Now, we are ready to define our pytorch model. This is done by creating a class that extends nn.Module, with two methods:

* __init__ : used to define and initialize the parameters of the model (for the LR, this is just a weight matrix of size 28*28 x 10, since we want to project to a 10-dimensional space to predict the digit. We could have define this explicitly, but with pytorch we have nn.Linear which is a shorthand for a linear projection.

* forward: used to define the computation in the model. We just want to apply the linear projection to the input, and then the softmax transformation (so that the output of the model can be interpreted as a probability distribution over the 10 digits).

Two things to note:

1. Since the images are tensors of size 28x28, and the Linear operator expects a 1-d array (since it is the operation Wx), we need to flatten the image using the method view (as a reshape).
2. Since the loss function is nll_loss (see next cell), it expects the input to be in log space, thats why we use log_softmax instead of softmax (see https://pytorch.org/docs/stable/nn.functional.html#log-softmax)

In [51]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.lin = nn.Linear(im_size, 10)

    def forward(self, x):
        x_flat = x.view(-1, im_size)
        return F.log_softmax(self.lin(x_flat), dim=1)

Next, we define the training loop.
For each minibatch (obtained using enumerate(train_loader) (see later for how the dataset is generated), we do the following:

1. Move x, y (data, target) to the GPU if necessary (using .to(device))
2. Reset the gradients to zero (from previous iterations) using .zero_grad()
3. Forward pass: we compute the predictions using model(data) (which is y = f(x) with f being the logistic regression)
4. Backward pass: we compute the gradients using .backward(), and then we apply one step of the optimizer, that is performing $w \leftarrow w - lr * \nabla (loss(f(x), y)$ for the case of gradient descent.

In [42]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

We can also create a function to evaluate our model. It is very similar to the previous one, but the main differences are:
    
1. We use torch.no_grad() to prevent gradient computations and changing the weights of the model.
2. Once we have the predicted distribution, we use .argmax to compute the most probable digit. We also compute the accuracy of our model

In [53]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Now, we are going to load the dataset. Since the MNIST dataset is very popular, PyTorch already has it included into datasets.MNIST, and will automatically download it the first time. We can also specify some transformations, like converting the matrix to a tensor and normalize the pixels

Note that we create two data loaders: one for the training set and another one for the testing set. Since this is just a quick experiment, we are not going to validate the initial hyperparameters, so we don't create another data loader for a validation set.

The main benefit of the DataLoader class is that it creates an object which can be used in a for loop, as we did in the previous train function, to iterate over the minibatches.

In [45]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

Now, we are almost done! We just instantiate the model, and tell which optimizer are we going to use, in this case, stochastic gradient descent

In [55]:
model = LogisticRegression().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

And finally, a simple loop over the number of epochs. This should take over a few minutes in a simple laptop cpu..

In [47]:
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.502584
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.293656
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.008645
Train Epoch: 1 [960/60000 (2%)]	Loss: 0.716611
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.413669
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.635522
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.499760
Train Epoch: 1 [2240/60000 (4%)]	Loss: 0.438110
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.766945
Train Epoch: 1 [2880/60000 (5%)]	Loss: 0.444802
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.273846
Train Epoch: 1 [3520/60000 (6%)]	Loss: 0.401564
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.516724
Train Epoch: 1 [4160/60000 (7%)]	Loss: 0.339985
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.304868
Train Epoch: 1 [4800/60000 (8%)]	Loss: 0.369957
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.469298
Train Epoch: 1 [5440/60000 (9%)]	Loss: 0.477505
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.373625
Train Epoch: 1 [6080/60000 (10%)]	Loss: 0.302387
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.33

Train Epoch: 1 [53120/60000 (89%)]	Loss: 0.460817
Train Epoch: 1 [53440/60000 (89%)]	Loss: 0.607142
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.333185
Train Epoch: 1 [54080/60000 (90%)]	Loss: 0.377787
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.292733
Train Epoch: 1 [54720/60000 (91%)]	Loss: 0.269112
Train Epoch: 1 [55040/60000 (92%)]	Loss: 0.175779
Train Epoch: 1 [55360/60000 (92%)]	Loss: 0.338969
Train Epoch: 1 [55680/60000 (93%)]	Loss: 0.063799
Train Epoch: 1 [56000/60000 (93%)]	Loss: 0.145126
Train Epoch: 1 [56320/60000 (94%)]	Loss: 0.196299
Train Epoch: 1 [56640/60000 (94%)]	Loss: 0.270612
Train Epoch: 1 [56960/60000 (95%)]	Loss: 0.212820
Train Epoch: 1 [57280/60000 (95%)]	Loss: 0.427648
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.316233
Train Epoch: 1 [57920/60000 (97%)]	Loss: 0.420929
Train Epoch: 1 [58240/60000 (97%)]	Loss: 0.607719
Train Epoch: 1 [58560/60000 (98%)]	Loss: 0.338059
Train Epoch: 1 [58880/60000 (98%)]	Loss: 0.603621
Train Epoch: 1 [59200/60000 (99%)]	Loss: 0.295032


Train Epoch: 2 [45760/60000 (76%)]	Loss: 0.170259
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.542554
Train Epoch: 2 [46400/60000 (77%)]	Loss: 0.349820
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.493208
Train Epoch: 2 [47040/60000 (78%)]	Loss: 0.640732
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.236006
Train Epoch: 2 [47680/60000 (79%)]	Loss: 0.214952
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.259743
Train Epoch: 2 [48320/60000 (81%)]	Loss: 0.177844
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.780076
Train Epoch: 2 [48960/60000 (82%)]	Loss: 0.489065
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.628844
Train Epoch: 2 [49600/60000 (83%)]	Loss: 0.317133
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.207148
Train Epoch: 2 [50240/60000 (84%)]	Loss: 0.078431
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.332951
Train Epoch: 2 [50880/60000 (85%)]	Loss: 0.217010
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.588351
Train Epoch: 2 [51520/60000 (86%)]	Loss: 0.367774
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.467980


Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.088655
Train Epoch: 3 [38720/60000 (65%)]	Loss: 0.359383
Train Epoch: 3 [39040/60000 (65%)]	Loss: 0.367778
Train Epoch: 3 [39360/60000 (66%)]	Loss: 0.231629
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.709220
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.082467
Train Epoch: 3 [40320/60000 (67%)]	Loss: 0.303909
Train Epoch: 3 [40640/60000 (68%)]	Loss: 0.185709
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.388017
Train Epoch: 3 [41280/60000 (69%)]	Loss: 0.755438
Train Epoch: 3 [41600/60000 (69%)]	Loss: 0.368632
Train Epoch: 3 [41920/60000 (70%)]	Loss: 0.276645
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.145981
Train Epoch: 3 [42560/60000 (71%)]	Loss: 0.194054
Train Epoch: 3 [42880/60000 (71%)]	Loss: 0.359890
Train Epoch: 3 [43200/60000 (72%)]	Loss: 0.230286
Train Epoch: 3 [43520/60000 (73%)]	Loss: 0.161422
Train Epoch: 3 [43840/60000 (73%)]	Loss: 0.196960
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.092242
Train Epoch: 3 [44480/60000 (74%)]	Loss: 0.370908


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.233041
Train Epoch: 4 [31040/60000 (52%)]	Loss: 0.504600
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.974269
Train Epoch: 4 [31680/60000 (53%)]	Loss: 0.298278
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.112368
Train Epoch: 4 [32320/60000 (54%)]	Loss: 0.346443
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.381974
Train Epoch: 4 [32960/60000 (55%)]	Loss: 0.088708
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.213965
Train Epoch: 4 [33600/60000 (56%)]	Loss: 0.077749
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.561460
Train Epoch: 4 [34240/60000 (57%)]	Loss: 0.272904
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.371713
Train Epoch: 4 [34880/60000 (58%)]	Loss: 0.304112
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.231458
Train Epoch: 4 [35520/60000 (59%)]	Loss: 0.110003
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.173806
Train Epoch: 4 [36160/60000 (60%)]	Loss: 0.272816
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.140243
Train Epoch: 4 [36800/60000 (61%)]	Loss: 0.235335


...

Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.821321
Train Epoch: 7 [8000/60000 (13%)]	Loss: 0.182077
Train Epoch: 7 [8320/60000 (14%)]	Loss: 0.094637
Train Epoch: 7 [8640/60000 (14%)]	Loss: 0.348598
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.252011
Train Epoch: 7 [9280/60000 (15%)]	Loss: 0.367067
Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.549473
Train Epoch: 7 [9920/60000 (17%)]	Loss: 0.210208
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.281764
Train Epoch: 7 [10560/60000 (18%)]	Loss: 0.701502
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.203423
Train Epoch: 7 [11200/60000 (19%)]	Loss: 0.209709
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.248001
Train Epoch: 7 [11840/60000 (20%)]	Loss: 0.198611
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.466947
Train Epoch: 7 [12480/60000 (21%)]	Loss: 0.251661
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.347769
Train Epoch: 7 [13120/60000 (22%)]	Loss: 0.332554
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.191988
Train Epoch: 7 [13760/60000 (23%)]	Loss: 0.243748
Train Ep


Test set: Average loss: 0.2719, Accuracy: 9214/10000 (92%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.557885
Train Epoch: 8 [320/60000 (1%)]	Loss: 1.044519
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.116339
Train Epoch: 8 [960/60000 (2%)]	Loss: 0.318695
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.218556
Train Epoch: 8 [1600/60000 (3%)]	Loss: 0.184906
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.404614
Train Epoch: 8 [2240/60000 (4%)]	Loss: 0.098455
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.252388
Train Epoch: 8 [2880/60000 (5%)]	Loss: 0.275054
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.327287
Train Epoch: 8 [3520/60000 (6%)]	Loss: 0.143427
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.385468
Train Epoch: 8 [4160/60000 (7%)]	Loss: 0.278824
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.564069
Train Epoch: 8 [4800/60000 (8%)]	Loss: 0.240557
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.230039
Train Epoch: 8 [5440/60000 (9%)]	Loss: 0.193551
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.285655
Train Epoch: 8 [6080/60000 (10%)

Train Epoch: 8 [53120/60000 (89%)]	Loss: 1.075200
Train Epoch: 8 [53440/60000 (89%)]	Loss: 0.249484
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.355575
Train Epoch: 8 [54080/60000 (90%)]	Loss: 0.202868
Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.290789
Train Epoch: 8 [54720/60000 (91%)]	Loss: 0.112977
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.140353
Train Epoch: 8 [55360/60000 (92%)]	Loss: 0.113326
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.105638
Train Epoch: 8 [56000/60000 (93%)]	Loss: 0.276465
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.061389
Train Epoch: 8 [56640/60000 (94%)]	Loss: 0.408982
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.446019
Train Epoch: 8 [57280/60000 (95%)]	Loss: 0.148079
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.166501
Train Epoch: 8 [57920/60000 (97%)]	Loss: 0.197524
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.087725
Train Epoch: 8 [58560/60000 (98%)]	Loss: 0.327259
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.241620
Train Epoch: 8 [59200/60000 (99%)]	Loss: 0.106409


Train Epoch: 9 [45760/60000 (76%)]	Loss: 0.109516
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.347990
Train Epoch: 9 [46400/60000 (77%)]	Loss: 0.146029
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.361872
Train Epoch: 9 [47040/60000 (78%)]	Loss: 0.172176
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.348224
Train Epoch: 9 [47680/60000 (79%)]	Loss: 0.252094
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.332306
Train Epoch: 9 [48320/60000 (81%)]	Loss: 0.185149
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.235317
Train Epoch: 9 [48960/60000 (82%)]	Loss: 0.404048
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.291109
Train Epoch: 9 [49600/60000 (83%)]	Loss: 0.165103
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.089935
Train Epoch: 9 [50240/60000 (84%)]	Loss: 0.267500
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.329087
Train Epoch: 9 [50880/60000 (85%)]	Loss: 0.105192
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.064268
Train Epoch: 9 [51520/60000 (86%)]	Loss: 0.213673
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.252822


Train Epoch: 10 [38080/60000 (63%)]	Loss: 0.622408
Train Epoch: 10 [38400/60000 (64%)]	Loss: 0.193231
Train Epoch: 10 [38720/60000 (65%)]	Loss: 0.228189
Train Epoch: 10 [39040/60000 (65%)]	Loss: 0.097684
Train Epoch: 10 [39360/60000 (66%)]	Loss: 0.369762
Train Epoch: 10 [39680/60000 (66%)]	Loss: 0.131939
Train Epoch: 10 [40000/60000 (67%)]	Loss: 0.308562
Train Epoch: 10 [40320/60000 (67%)]	Loss: 0.325334
Train Epoch: 10 [40640/60000 (68%)]	Loss: 0.310182
Train Epoch: 10 [40960/60000 (68%)]	Loss: 0.055145
Train Epoch: 10 [41280/60000 (69%)]	Loss: 0.071138
Train Epoch: 10 [41600/60000 (69%)]	Loss: 0.638136
Train Epoch: 10 [41920/60000 (70%)]	Loss: 0.206017
Train Epoch: 10 [42240/60000 (70%)]	Loss: 0.356118
Train Epoch: 10 [42560/60000 (71%)]	Loss: 0.178917
Train Epoch: 10 [42880/60000 (71%)]	Loss: 0.177836
Train Epoch: 10 [43200/60000 (72%)]	Loss: 0.097855
Train Epoch: 10 [43520/60000 (73%)]	Loss: 0.393157
Train Epoch: 10 [43840/60000 (73%)]	Loss: 0.130316
Train Epoch: 10 [44160/60000 (7

As our first attemp, we obtained over 92% accuracy, which is not so bad, but soon we will see how to vastly improve on this result!